In [1]:
import dpeeg

dpeeg.set_seed()
print(dpeeg.__version__)

0.3.5


In [2]:
from dpeeg.data import datasets, transforms

trans = transforms.ComposeTransforms(
    transforms.Normalization(),
    transforms.Augmentation('segmentation_and_reconstruction', multiply=1),
    transforms.Unsqueeze()
)

dataset = datasets.BCICIV2A(subjects=[1], transforms=trans)

Reading BCICIV 2A Dataset ...


Tensorflow not install, you could not use those pipelines
To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


Create sub1 Epochs ...


48 events found
Event IDs: [1 2 3 4]
Not setting metadata
48 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 48 events and 1001 original time points ...
0 bad epochs dropped
48 events found
Event IDs: [1 2 3 4]
Not setting metadata
48 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 48 events and 1001 original time points ...
0 bad epochs dropped
48 events found
Event IDs: [1 2 3 4]
Not setting metadata
48 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 48 events and 1001 original time points ...
0 bad epochs dropped
48 events found
Event IDs: [1 2 3 4]
Not setting metadata
48 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 48 events and 1001 original time points ...
0 bad epochs dropped
48 events found
Event IDs: [1 2 

[Normalization(mode=z-score, factor_new=0.001, verbose=None) starting] ...
[Augmentation(method=segmentation_and_reconstruction, only_train=True, kwargs={'multiply': 1}) starting] ...
[Unsqueeze(dim=1) starting] ...
[Loading dataset done]


In [3]:
dataset[1]['train']

(array([[[[ 0.76894367,  0.70663553, -0.20402197, ..., -0.61142138,
           -0.35739587, -0.39094641],
          [ 0.91752463,  0.77373661,  0.11710462, ..., -0.64017899,
           -0.65935072, -0.17526437],
          [ 0.84083769,  0.87438823,  0.05479647, ..., -0.91816917,
           -0.67852246, -0.5299415 ],
          ...,
          [-0.38136054, -0.06981982, -0.3621888 , ..., -1.14823001,
           -1.19615936, -0.87503276],
          [-0.06023395,  0.14106929, -0.1944361 , ..., -1.11947241,
           -0.93254797, -0.6833154 ],
          [-0.24715838, -0.00271874, -0.07461275, ..., -0.59224965,
           -0.75520941, -0.56828497]]],
 
 
        [[[ 0.97503985,  1.50705554,  2.26433915, ...,  0.15065516,
            0.50053936,  0.33757959],
          [ 1.37285339,  1.55977782,  2.15410166, ..., -0.10337035,
            0.09793288,  0.08834701],
          [ 1.38723219,  1.88569735,  2.58546574, ..., -0.36698174,
           -0.08419862,  0.03562474],
          ...,
          